This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5) and depends on Magma (tested using version
2.26-9).

In this notebook, we make an intermediate calculation towards finding generic curves of genus 6 which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1.  Allow 20 minutes for completion.

In [2]:
load("preamble.sage")

Construct the Plucker relations defining $G(2,5)$ inside $\mathbb{P}^9$, then verify that these relations vanish on the points we have constructed.

In [3]:
F = GF(2)

In [4]:
P.<x01,x02,x03,x04,x12,x13,x14,x23,x24,x34> = PolynomialRing(F, 10)
quads = (x01*x23 + x02*x13 + x03*x12,
         x01*x24 + x02*x14 + x04*x12,
         x01*x34 + x03*x14 + x04*x13,
         x02*x34 + x03*x24 + x04*x23,
         x12*x34 + x13*x24 + x14*x23)

Construct generators for $\mathrm{GL}(5, \mathbb{F}_2)$ acting on $\wedge^2 \mathbb{F}_2^5$.

In [5]:
G0 = SymmetricGroup(10)
g1 = G0([(1,5,8,10,4),(2,6,9,3,7)]).matrix().change_ring(F)
g2 = identity_matrix(F, 10)
g2[1,4] = 1
g2[2,5] = 1
g2[3,6] = 1
G = MatrixGroup([g1,g2])

Construct an orbit lookup tree of depth 4.

In [6]:
def vecs_to_gens(vecs):
    return tuple(sum(P.gens()[i] * v[i] for i in range(10)) for v in vecs)

In [7]:
def linear_section(coords):
    V = Matrix(coords).right_kernel()
    tmp2 = vecs_to_gens(V.gens()) + quads
    return P.ideal(tmp2)

In [8]:
def apply_group_elem(g, x):
    x1 = g*x
    x1.set_immutable()
    return x1

In [9]:
def stabilizer(x):
    return vec_stab(Matrix(F, x), transpose=True)

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
def forbid(vecs, easy=False):
    if Matrix(vecs).rank() < len(vecs):
        return True
    return False

In [12]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep,
           'forbid': forbid}

In [13]:
S = [x for x in VectorSpace(F, 10) if x != 0]
for x in S:
    x.set_immutable()

In [14]:
tree = build_orbit_tree(G, S, 4, methods, verbose=False)

Identify 4-tuples whose spans are $G$-equivalent.

In [16]:
gens = [GL(4,F).random_element().matrix() for _ in range(10)]
edges = []
for mats in green_nodes(tree, 4):
    for g in gens:
        mats0 = tuple(sum(g[i,j]*mats[j] for j in range(4)) for i in range(4))
        for v in mats0:
            v.set_immutable()
        mats1, g1 = orbit_rep_from_tree(G, tree, mats0, apply_group_elem, optimized_rep)
        edges.append((mats, mats1))

In [17]:
Gamma = Graph(edges, loops=True)
l = [cc[0] for cc in Gamma.connected_components(sort=False)]
len(l)

57

Write coordinates of representative tuples to a file.

In [19]:
with open("genus6-flats.txt", "w") as f:
    f.write(str(l))